In [ ]:
# Used Etherscan.io APIs"

In [ ]:
import requests
import pandas as pd
import numpy as np
import sys
import time
import os
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
pd.set_option('display.max_rows', 200)

In [ ]:
### Load List of Contracts

In [ ]:
contract = pd.read_csv("data/dapps_with_licences_2020-12-09.csv")

In [ ]:
pd.DataFrame(contract.license.value_counts()).to_csv('licences.csv')

In [ ]:
contract = contract[contract.license!='Proprietary']
contract = contract[contract.license!='Proprietry']
contract = contract[-contract.license.isna()]
contract = contract[contract.license!='Private']
contract = contract[contract.license!='proprietary']
contract = contract[contract.license!='Proprietary (May change later as we publish)']
contract = contract[contract.license!='Company Registration']
contract = contract[contract.license!='Unknown licensen']
contract = contract[contract.license!='Proprietary/MIT']
contract = contract[contract.license!='TBD']
contract = contract[contract.license!='Copyright']
contract = contract[contract.license!='No License']
contract = contract[-contract.license.str.contains('agpl', case=False)]
contract = contract[-contract.license.str.contains('affero', case=False)]
print(contract.shape)

In [ ]:
mit_contract = contract[contract.license.str.contains('mit', case=False)]
open_contract = contract[contract.license.str.contains('open', case=False)]
free_contract = contract[contract.license.str.contains('free', case=False)]
un_contract = contract[contract.license.str.contains('unlicen', case=False)]
apache_contract = contract[contract.license.str.contains('apache', case=False)]
gpl_contract = contract[contract.license.str.contains('gpl', case=False)]
bsd_contract = contract[contract.license.str.contains('bsd', case=False)]
gnu_contract = contract[contract.license.str.contains('GNU General Public License', case=False)]

In [ ]:
os_contracts = pd.concat([mit_contract, open_contract, free_contract, un_contract, apache_contract, gpl_contract, bsd_contract, gnu_contract])
os_contracts = os_contracts[-os_contracts.license.str.contains('proprietary', case=False)]
os_contracts = os_contracts[-os_contracts.license.str.contains('ETH.Limited', case=False)]
os_contracts = os_contracts[-os_contracts.license.str.contains('Towards OpenSource/MIT, but work in progress', case=False)]
print(os_contracts.shape)

In [ ]:
# contants
with open("token.txt") as file:  
    API_Token = file.read() 

In [ ]:
os_contracts.to_csv('data/dapps_open_source.csv')

In [ ]:
ADDRESS_ARRAY = os_contracts.hash.values

### Create Folders

In [ ]:
sol_source = 'data/sol_source_open_source'
not_verified = 'data/sol_source_not_verified_open_source'

In [ ]:
if not os.path.exists(sol_source):
    os.mkdir(sol_source)
    
if not os.path.exists(not_verified):
    os.mkdir(not_verified)

In [ ]:
files_in_sol = os.listdir(sol_source)
files_in_sol = [i.split('_')[0] for i  in files_in_sol]
files_in_not_ver = os.listdir(not_verified)
files_in_not_ver = [i.split('_')[0] for i  in files_in_not_ver]
files_in_sol.extend(files_in_not_ver)
files_in_sol = set(files_in_sol)
not_extracted = list(set(ADDRESS_ARRAY) - files_in_sol)

In [ ]:
def scrape_ether_contract_and_write(address_array, API_Token):
    c = 0
    for i, address in tqdm(enumerate(address_array)): 
        # time.sleep(0.01) # we can do 5 GET/POST requests per sec
        url = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={address}&apikey={API_Token}'
        resp = requests.get(url=url)
        data = resp.json()
        try:
            contract_name = data['result'][0]['ContractName']

            # save solidity source code
            with open(f"{sol_source}/{address}_{contract_name}.sol", "w", encoding="utf-8") as f:
                print(data['result'][0]['SourceCode'], file=f)

        except:
            c += 1
            time.sleep(0.25)
                    
    print(f'did not work {c} times')

In [ ]:
scrape_ether_contract_and_write(address_array=not_extracted, API_Token=API_Token)